In [1]:
from cirq_sic import *

import numpy as np
np.set_printoptions(precision=3, suppress=True)

import bqskit as bq
from bqskit.ext import bqskit_to_cirq, cirq_to_bqskit

In [2]:
bqskit_willow_gateset = [bq.ir.gates.PhasedXZGate(), # cirq.ops.phased_x_z_gate.PhasedXZGate
                         bq.ir.gates.RXGate(), # cirq.ops.phased_x_gate.PhasedXPowGate
                         bq.ir.gates.IdentityGate(), # cirq.I
                         #bq.ir.gates.MeasurementPlaceholder(), #cirq.ops.measurement_gate.MeasurementGate
                         bq.ir.gates.CZGate(), # cirq.CZ
                         #bq.ir.gates.BarrierPlaceholder(), # cirq.ops.wait_gate.WaitGate
                         bq.ir.gates.HGate(), # cirq.ops.common_gates.HPowGate
                         bq.ir.gates.RYGate(), # cirq.ops.common_gates.YPowGate
                         bq.ir.gates.Reset(), #  cirq.ops.common_channels.ResetChannel
                         bq.ir.gates.SGate(), # cirq.ops.clifford_gate.SingleQubitCliffordGate
                         bq.ir.gates.XGate(), # cirq.ops.clifford_gate.SingleQubitCliffordGate
                         bq.ir.gates.YGate(), # cirq.ops.clifford_gate.SingleQubitCliffordGate
                         bq.ir.gates.ZGate(), # cirq.ops.clifford_gate.SingleQubitCliffordGate  
                         bq.ir.gates.RZGate(), # cirq.ops.common_gates.ZPowGate
                         #cirq.ops.global_phase_op.GlobalPhaseGate
                         #bq.ir.gates.RXGate() # cirq.ops.common_gates.XPowGate
]

def optimize_circuit(qubits, circuit, processor_id="willow_pink", optimization_level=1):
    device = cirq_google.engine.create_device_from_processor_id(processor_id)
    gateset = device.metadata.compilation_target_gatesets[0]
    connectivity_graph = device.metadata.nx_graph

    qubit_index = dict([(q, i) for i, q in enumerate(device.metadata.qubit_set)])
    qubit_pairs = list(device.metadata.nx_graph.edges)
    full_coupling_graph = [(qubit_index[a], qubit_index[b]) for a, b in qubit_pairs]
    used_indices = [qubit_index[qubit] for qubit in qubits]
    restricted_coupling_graph = [pair for pair in full_coupling_graph if pair[0] in used_indices and pair[1] in used_indices]
    coupling_graph = [(used_indices.index(pair[0]), used_indices.index(pair[1])) for pair in restricted_coupling_graph]

    bq_circuit = cirq_to_bqskit(circuit)
    model = bq.MachineModel(bq_circuit.num_qudits, gate_set=bqskit_willow_gateset, coupling_graph=coupling_graph)
    compiled_bq_circuit, initial_mapping, final_mapping = bq.compile(bq_circuit, model=model, optimization_level=optimization_level, with_mapping=True)

    compiled_circuit = bqskit_to_cirq(compiled_bq_circuit)
    named_qubits = [cirq.NamedQubit("q_%d" % i) for i in range(len(qubits))]
    qubit_map = dict([(nq, qubits[i]) for i, nq in enumerate(named_qubits)])

    finished_circuit = cirq.drop_negligible_operations(\
                       cirq.drop_terminal_measurements(\
                            compiled_circuit.transform_qubits(qubit_map)))
    final_measurement = None
    for op in circuit.all_operations():
        if isinstance(op.gate, cirq.MeasurementGate):
            final_measurement = op
            break 
    finished_circuit.append(final_measurement)
    device.validate_circuit(finished_circuit)
    return finished_circuit, final_mapping

def results_to_freqs(results, mapping=None):
    if mapping is not None:
        measurements = results.measurements["result"][:, mapping]
    else:
        measurements = results.measurements["result"]
    n_outcomes = 2**measurements.shape[1]
    int_outcomes = [big_endian_bits_to_int(bits) for bits in measurements]
    counts = collections.Counter(int_outcomes)
    for i in range(n_outcomes):
        if i not in counts:
            counts[i] = 0
    freqs = np.array([v for k, v in sorted(counts.items())])/counts.total()
    return freqs

In [3]:
n = 2
d = 2**n
phi = load_sic_fiducial(d)
ket = rand_ket(d)

qubits = cirq.GridQubit.rect(2, n, top=5, left=1)
system_qubits, ancilla_qubits = qubits[:n], qubits[n:]
prepare_fiducial = ansatz_circuit(phi)
prepare_system = ansatz_circuit(ket)
circuit = cirq.Circuit((prepare_system(system_qubits),\
                        simple_wh_povm(system_qubits, ancilla_qubits, prepare_fiducial=prepare_fiducial, measure=True)))

In [4]:
optimized_circuit, mapping = optimize_circuit(qubits, circuit, optimization_level=4)
get_gate_counts(optimized_circuit)

/Users/heyredhat/miniforge3/envs/cirq_sic/lib/python3.13/site-packages/bqskit/passes/synthesis/leap.py:358: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  m, y_int, _, _, _ = linregress(best_layers, best_dists)
/Users/heyredhat/miniforge3/envs/cirq_sic/lib/python3.13/site-packages/bqskit/passes/synthesis/leap.py:358: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  m, y_int, _, _, _ = linregress(best_layers, best_dists)


--- Gate Counts (by type) ---
Rz: 27
Rx: 21
CZPowGate: 12
MeasurementGate: 1


In [5]:
E = wh_povm(phi)
p = change_conjugate_convention(np.array([ket.conj() @ e @ ket for e in E]).real)

result = cirq.Simulator().simulate(cirq.drop_terminal_measurements(optimized_circuit),\
                                   qubit_order=[qubits[i] for i in mapping])
assert np.allclose(p, abs(result.final_state_vector)**2)

In [6]:
N = 100000
s = cirq.Simulator()
results = s.run(optimized_circuit, repetitions=N)
results_to_freqs(results, mapping=mapping), p

(array([0.123, 0.008, 0.016, 0.076, 0.17 , 0.134, 0.094, 0.044, 0.034,
        0.06 , 0.045, 0.009, 0.012, 0.097, 0.009, 0.069]),
 array([0.123, 0.008, 0.017, 0.078, 0.169, 0.136, 0.093, 0.045, 0.034,
        0.06 , 0.044, 0.009, 0.012, 0.096, 0.009, 0.069]))

## Compare

In [7]:
device = cirq_google.engine.create_device_from_processor_id("willow_pink")
gateset = device.metadata.compilation_target_gatesets[0]
connectivity_graph = device.metadata.nx_graph
alternate = process_circuit(circuit, connectivity_graph, gateset, qubits)
device.validate_circuit(alternate)
get_gate_counts(alternate)

--- Gate Counts (by type) ---
PhasedXPowGate: 31
CZPowGate: 21
ZPowGate: 4
PhasedXZGate: 13
MeasurementGate: 1


In [ ]:
#RandomDevicePlacer
# Now synthesize with a cached Compiler
#CIRCUITS = "../experiments/circuits/"
#with open(CIRCUITS+"input.qasm", "w") as f:
#    f.write(cirq.qasm(circ)) # args=cirq.QasmArgs(version="3.0"))